In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

MODEL_NAME = "IlyaGusev/saiga_mistral_7b"
DEFAULT_MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>"
DEFAULT_RESPONSE_TEMPLATE = "<s>bot\n"
DEFAULT_SYSTEM_PROMPT = "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."


class Conversation:
    def __init__(
        self,
        message_template=DEFAULT_MESSAGE_TEMPLATE,
        system_prompt=DEFAULT_SYSTEM_PROMPT,
        response_template=DEFAULT_RESPONSE_TEMPLATE,
    ):
        self.message_template = message_template
        self.response_template = response_template
        self.messages = [{"role": "system", "content": system_prompt}]

    def add_user_message(self, message):
        self.messages.append({"role": "user", "content": message})

    def add_bot_message(self, message):
        self.messages.append({"role": "bot", "content": message})

    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += DEFAULT_RESPONSE_TEMPLATE
        return final_text.strip()


def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(**data, generation_config=generation_config)[0]
    output_ids = output_ids[len(data["input_ids"][0]) :]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()


config = PeftConfig.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    # load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
model = PeftModel.from_pretrained(model, MODEL_NAME, torch_dtype=torch.float16)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
print(generation_config)

inputs = [
    "Почему трава зеленая?",
    # "Сочини длинный рассказ, обязательно упоминая следующие объекты. Дано: Таня, мяч",
]
for inp in inputs:
    conversation = Conversation()
    conversation.add_user_message(inp)
    prompt = conversation.get_prompt(tokenizer)

    output = generate(model, tokenizer, prompt, generation_config)
    print(inp)
    print(output)
    print()
    print("==============================")
    print()

/home/kosenko/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:29<00:00, 14.53s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_new_tokens": 1536,
  "no_repeat_ngram_size": 15,
  "pad_token_id": 0,
  "repetition_penalty": 1.1,
  "temperature": 0.2,
  "top_k": 40,
  "top_p": 0.9
}

Почему трава зеленая?
Вопрос о цвете растений является сложным и многоаспектным. Зеленый цвет у большинства растений обусловлен наличием в них хлорофилла - пигмента, который позволяет растениям совершать фотосинтез. Фотосинтез - это процесс, благодаря которому растения получают энергию из света и преобразуют её в химическую энергию, используя углекислый газ из воздуха и воду. Хлорофилл поглощает световые волны длиной 430-450 нм (синий цвет) и 670-680 нм (красный цвет), что позволяет растениям использовать максимальное количество энергии из света.

Хлорофилл имеет структуру, состоящую из двух молекул пигмента, которые образуют димер. Внутри каждой молекулы хлорофилла есть две группы, которые отражают свет: карбоксильные группы и метильные группы. К

In [24]:
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import json
from pathlib import Path
from functools import partial
from datasets import load_dataset
import pandas as pd


class Prompts:
    def rucola_prompt(self, item):
        sentence1 = item["sentence"]
        prompt = f"""Предложение: {sentence1}\nКорректно ли данное предложение с точки зрения русского языка? Ответь только "да" или "нет"."""
        return prompt


class Eval:
    def rucola_eval(
        self,
        item=None,
        result: str = None,
    ):
        answer = None
        result = result.lower()
        if "да" in result:
            answer = 1
        elif "не" in result:
            answer = 0
        else:
            answer = int(not bool(item["label"]))
        return answer


class Models:
    # @staticmethod
    # def verbalist_generation_1(prompt=None, model=None):
    #     conversation = VerbalistConversation()
    #     conversation.add_user_message(prompt)
    #     prompt = conversation.get_prompt(tokenizer)
    #     # print("PROMPT", prompt)
    #     generation_config = GenerationConfig(
    #         bos_token_id=1,
    #         eos_token_id=2,
    #         pad_token_id=0,
    #         max_new_tokens=512,
    #         # no_repeat_ngram_size=15,
    #         repetition_penalty=1.1,
    #         temperature=0.5,
    #         top_k=40,
    #         top_p=0.95,
    #         # do_sample=True,
    #     )
    #     output = generate(
    #         model,
    #         tokenizer,
    #         prompt,
    #         generation_config,
    #     )
    #     # print("RESULT", output)
    #     return output

    @staticmethod
    def saiga_mistral(prompt=None, model=None):
        conversation = Conversation()
        conversation.add_user_message(prompt)
        prompt = conversation.get_prompt(tokenizer)

        output = generate(model, tokenizer, prompt, generation_config)

        return output

    def dummy(self, prompt=None):
        tasks_map = {
            "rucola": "да",
        }
        return tasks_map[self.dataset_name]


class EvalRussianSuperGlue(
    Prompts,
    Eval,
    Models,
):
    def __init__(
        self,
        dataset_name="rucola",
        model_type=None,
        base_folder=None,
        generation_function=None,
        split=None,
        eval_name=None,
        debug_mode=False,
    ) -> None:
        self.dataset_name = dataset_name
        self.dataset = load_dataset("RussianNLP/rucola")
        self.split = split

        self.dataset = self.dataset[self.split]

        self.generation_function = generation_function
        self.model_type = model_type

        self.base_folder = Path(base_folder)
        self.eval_name = eval_name

        self.debug_mode = debug_mode

        if self.debug_mode:
            num = 10
            self.dataset = self.dataset.select(range(20, 20 + num))

        self.eval_filenames = {
            "rucola": "RuCoLA",
        }

    def evaluate(self):
        task_name = self.eval_filenames[self.dataset_name]
        print(task_name)

        eval_folder = self.base_folder / f"{self.eval_name}" / self.split
        eval_folder.mkdir(exist_ok=True, parents=True)
        output_file = eval_folder / f"{task_name}.csv"

        if output_file.is_file() and not self.debug_mode and self.model_type != "dummy":
            with open(eval_folder / f"{task_name}.txt", "r") as f:
                print(f.read())
        else:
            predicts = []
            ground_true = []

            with open(eval_folder / f"{task_name}.log", "w") as f:
                idxs = []

                for item in tqdm(self.dataset):
                    prompt = self.get_prompt(item=item)
                    result = self.get_answer(prompt=prompt)

                    print(prompt, file=f)
                    print(f"predict answer = {result}", file=f)
                    print(f"real answer = {item['label']}", file=f)

                    answer = self.evaluate_answer(item=item, result=result)
                    gold_true = self.get_gold_true(item=item)

                    predicts.append(answer)
                    ground_true.append(gold_true)
                    idxs.append(item["id"])

                acc = str(accuracy_score(ground_true, predicts))

                print(f"Accuracy: {acc}")

                # with open(output_file, "w") as f:
                #     for idx, predict in zip(idxs, predicts):
                #         answer = {
                #             "id": idx,
                #             "acceptable": predict,
                #         }
                #         json.dump(answer, f, ensure_ascii=False)
                #         f.write("\n")
                pd.DataFrame(
                    data={
                        "id": idxs,
                        "acceptable": predicts,
                    }
                ).to_csv(output_file, index=False)

                with open(eval_folder / f"{task_name}.txt", "w") as f:
                    f.write(acc)

    def get_answer(self, prompt):
        answer = None
        if self.model_type == "dummy":
            answer = self.dummy(prompt=prompt)
        else:
            answer = self.generation_function(prompt=prompt)
        answer = answer.strip()
        return answer

    def get_gold_true(self, item):
        handlers_map = {
            "rucola": lambda item: item["label"],
        }
        return handlers_map[self.dataset_name](item=item)

    def get_prompt(self, item):
        handlers_map = {
            "rucola": self.rucola_prompt,
        }
        return handlers_map[self.dataset_name](item=item)

    def evaluate_answer(self, result, item):
        handlers_map = {
            "rucola": self.rucola_eval,
        }
        return handlers_map[self.dataset_name](item=item, result=result)


for name in [
    "rucola",
]:
    evaluation = EvalRussianSuperGlue(
        dataset_name=name,
        # split="validation",
        split="test",
        # model_type="dummy",
        base_folder="verbalist/evaluation/RuCoLA/valid",
        # eval_name="dummy",
        eval_name="saiga_mistral",
        # debug_mode=True,
        # generation_function=partial(
        #     RussianSuperGlueModels.chat_gpt,
        # ),
        generation_function=partial(Models.saiga_mistral, model=model),
    )

    evaluation.evaluate()

RuCoLA


100%|██████████| 2789/2789 [08:40<00:00,  5.36it/s]

Accuracy: 0.0
